In [11]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import sklearn.metrics as sk_metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import tempfile
import statsmodels.api as sm
import os
from tensorflow import keras
import statsmodels.formula.api as smf

from tensorflow.keras import layers
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
import pymssql
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from tqdm import tqdm
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from mpl_toolkits import mplot3d

# modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# for modeling
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

random_state = 2023

# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)

2.9.0


In [12]:
def get_column_index(cursor, column_name):
    for index, col in enumerate(cursor.description):
        if col[0] == column_name:
            return index
additional_data_headers = ["RentMortgagePayment" ,"TotalIncome" ,"RentMortgage" ,"CarPayment" ,"Expenses" ,"AvailableIncome" ,"Miles"]
additional_data_sql = "VIN,"
for i, header in enumerate(additional_data_headers):
    additional_data_sql += header
    if i!=len(additional_data_headers)-1:
        additional_data_sql+=','

In [43]:
conn = pymssql.connect(server='localhost', user='sa', password='Passw0rd!', database='Loans_2023_01_27')
cursor = conn.cursor()

headers = []
rows = []

# Underwriting Data
cursor.execute("SELECT * FROM dbo.UnderwritingData")

for entity in cursor.description:
    headers.append(entity[0])

vin_index = get_column_index(cursor, "VIN")

res = cursor.fetchone()
while res:
    row = [element for element in res]
    if row[vin_index] is not None:
        rows.append(row)
    res = cursor.fetchone()

df = pd.DataFrame(rows, columns=headers)
ids = [i for i in range(0, len(df))]
default_status = ["NA" for _ in range(len(df))]
default_delq = ["NA" for i in range(len(df))]
default_apr = ["NA" for _ in range(len(df))]
df["ID"] = ids
df["APR"] = default_apr
df["Status"] = default_status
df["Delq"] = default_delq
df = df[df['OpeningBalance']!=0]


#Raw algorithm data
headers = []
rawAlgRows =[]
cursor.execute("Select * FROM dbo.RawAlgorithmData")

for entity in cursor.description:
    headers.append(entity[0])

vin_index = get_column_index(cursor, "Vin6")

res = cursor.fetchone()
while res:
    row = [element for element in res]
    if row[vin_index] is not None:
        rawAlgRows.append(row)
    res = cursor.fetchone()

rawAlgDf = pd.DataFrame(rawAlgRows, columns=headers)
#prune extra space from end of vins
rawAlgDf['Vin6'] = rawAlgDf['Vin6'].apply(lambda x: x.strip())
df['VIN'] = df['VIN'].apply(lambda x: x.strip())


#matching by credit score
for idx, row in tqdm(rawAlgDf.iterrows()):
    vin = row["Vin6"]
    vinsFromUnder = df[df["VIN"]==vin]


    for index, match in vinsFromUnder.iterrows():
        matchID = match["ID"]
        underwritingCreditScore = (int)(match["CreaditScore"])
        rawCreditScore = (int)(row["Score"])

        #we don't want to match if the scores are zero, since that means they are missing
        if rawCreditScore==0:
            continue

        if underwritingCreditScore==rawCreditScore:
            df.loc[df["ID"]==matchID,"Status"]=row["ProcStatsCd"]
            df.loc[df["ID"]==matchID,"Delq"]=row["Delq"]
            df.loc[df["ID"]==matchID, "APR"] = int(row["Apr"] * 100) if row["Apr"] is not None else None #convert apr to an int

df = df[(df['Status']=='CO') | (df['Status']=='CL')]
df['Status'].replace(to_replace=['CO', 'CL'], value=[0,1], inplace=True)
df['HomeOwner'].replace(to_replace=['N', 'Y'], value=[0,1], inplace=True)
df['DelqPct'] = df.apply(lambda row : (100*row['Delq'])/row['OpeningBalance'],axis=1)
df.fillna(df.mean(), inplace=True)

#TODO maybe put back: CreaditScore, CurrentPaymentsMade, DelqPct
df = df[["OpeningBalance","Salary","Term","VehicleYear","LTV","DTI","TradelinesTotal","TradelinesBalance","Reposessions","Bankrupcies","Chageoff","Mortgage","NinetyDaysDelinquent","InquiresTotal","OpenTotal","ClosedTotalTermsNotPaid","LatePaymentHistory001230","LatePaymentHistory001260","LatePaymentHistory001290Plus","PaidAccounts","InquiriesLast6Months","OpenTotalInBadStatus","RentMortgage","CashDown","HomeOwner", "APR", "CreaditScore", "DelqPct", "Status"]]

#outlier handling
for col in df.columns:
    if(col in ["Status", "DelqPct", "CreaditScore", "HomeOwner", "Reposessions", "Chageoff", "APR"]):
        continue
    percentiles = df[col].quantile([0.00, 0.999]).values
    df.loc[df[col]<=percentiles[0], col] = percentiles[0]
    df.loc[df[col]>=percentiles[1], col] = percentiles[1]

26508it [00:34, 779.36it/s] 
/var/folders/kh/np0tdw052gg77h97rt6rm3l80000gn/T/ipykernel_71904/1059541156.py:81: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [56]:
from scipy.stats import pearsonr

corrs = dict()

for col in df.columns:
    if col == "DelqPct" or col=="Status":
        continue
    corr, _ = pearsonr(df[col], df["DelqPct"])
    corrs[col] = abs(corr)

corrs = {k: v for k, v in sorted(corrs.items(), key=lambda item: item[1])}

goodCols = []

for col, score in corrs.items():
    if score>=0.1:
        goodCols.append(col)

print(goodCols)
goodCols.append("Status")

dfCorr = df[goodCols]
# print(dfCorr)

X=np.array(dfCorr.iloc[:,:-1])
Y = dfCorr.iloc[:,-1:]

X_train, X_test, y_train, y_test = train_test_split(
    dfCorr.iloc[:,:-1], dfCorr.iloc[:,-1:], test_size=0.2, random_state=42)
print(X_train.shape)

['NinetyDaysDelinquent', 'OpenTotalInBadStatus', 'Salary', 'CreaditScore', 'CashDown', 'Term', 'APR', 'OpeningBalance']
(6721, 8)


In [57]:

# build a model
model = Sequential()
model.add(Dense(16, input_shape=(X.shape[1],), activation='relu')) # Add an input shape! (features,)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# compile the model
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# early stopping callback
# This callback will stop the training when there is no improvement in
# the validation loss for 10 consecutive epochs.
es = EarlyStopping(monitor='val_accuracy',
                                   mode='max', # don't minimize the accuracy!
                                   patience=10,
                                   restore_best_weights=True)

# now we just update our model fit call
history = model.fit(X,
                    Y,
                    callbacks=[es],
                    epochs=80, # you can set this to a big number!
                    batch_size=10,
                    validation_split=0.2,
                    shuffle=True,
                    verbose=1)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 16)                144       
                                                                 
 dense_10 (Dense)            (None, 16)                272       
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
Epoch 1/80


2023-04-04 15:35:35.664756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


673/673 [==============================] - ETA: 0s - loss: 15.9118 - accuracy: 0.5341

2023-04-04 15:35:42.357882: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


673/673 [==============================] - 8s 11ms/step - loss: 15.9118 - accuracy: 0.5341 - val_loss: 4.5705 - val_accuracy: 0.6306
Epoch 2/80
673/673 [==============================] - 7s 10ms/step - loss: 9.0070 - accuracy: 0.5269 - val_loss: 8.1163 - val_accuracy: 0.4557
Epoch 3/80
673/673 [==============================] - 7s 10ms/step - loss: 6.0096 - accuracy: 0.5276 - val_loss: 2.3333 - val_accuracy: 0.6151
Epoch 4/80
673/673 [==============================] - 7s 10ms/step - loss: 7.3351 - accuracy: 0.5248 - val_loss: 5.2147 - val_accuracy: 0.5158
Epoch 5/80
673/673 [==============================] - 5s 8ms/step - loss: 6.0681 - accuracy: 0.5269 - val_loss: 12.9662 - val_accuracy: 0.4598
Epoch 6/80
673/673 [==============================] - 5s 7ms/step - loss: 6.0743 - accuracy: 0.5282 - val_loss: 13.1593 - val_accuracy: 0.5407
Epoch 7/80
673/673 [==============================] - 5s 7ms/step - loss: 9.6086 - accuracy: 0.5225 - val_loss: 4.4268 - val_accuracy: 0.5199
Epoch 8/80